In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from time import time
from gensim.models import Word2Vec

In [2]:
m=Word2Vec([['good']*10,['bad']*10],vector_size=20,alpha=0.03,min_count=10,min_alpha=0.0007,window=4,hs=1,sample=0.00001)

In [3]:
print(m)

Word2Vec<vocab=2, vector_size=20, alpha=0.03>


In [4]:
from gensim.models import Word2Vec
init=Word2Vec.load('PLS_c10.model')

In [5]:
init

In [4]:
f = pd.read_csv('./yelp_review_polarity_csv/test.csv', header=None)  # , encoding='ISO-8859-1'
f.columns = ["label", "review"]
f.loc[f['label'] == 1, 'label'] = 0
f.loc[f['label'] == 2, 'label'] = 1

true_label = f.label
yelp_review = f.review

In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
import time
start_time=time.time()
clean_data=[remove_stopwords(review)for review in yelp_review]#remove frequent words that does not carry sentiment
tokenised_data = [simple_preprocess(line, deacc=True) for line in clean_data] #remove punctuations and lowercase words also tokenise them
print('time taken: '+str(time.time()-start_time))
print(tokenised_data[:1])

time taken: 4.889155864715576
[['contrary', 'reviews', 'zero', 'complaints', 'service', 'prices', 'getting', 'tire', 'service', 'past', 'years', 'now', 'compared', 'experience', 'places', 'like', 'pep', 'boys', 'guys', 'experienced', 'know', 'they', 're', 'doing', 'nalso', 'place', 'feel', 'like', 'taken', 'advantage', 'of', 'gender', 'other', 'auto', 'mechanics', 'notorious', 'capitalizing', 'ignorance', 'cars', 'sucked', 'bank', 'account', 'dry', 'but', 'here', 'service', 'road', 'coverage', 'explained', 'let', 'decide', 'nand', 'renovated', 'waiting', 'room', 'it', 'looks', 'lot', 'better', 'previous', 'years']]


In [12]:
tokenised_data=pd.Series(tokenised_data,name='tokenised_data')

In [62]:
# WARNING
from gensim.models import Word2Vec
# Skip-gram model (sg = 1)
# size = 20
# window = 3
# min_count = 1
# workers = 3


vector_size=20
alpha=0.03
min_count=10
min_alpha=0.0007
window=4
hs=1
sample=0.00001
epochs=30


word2vec_model_file =  'word2vec' + str(vector_size) + 'tokenised.model'
start_time = time()
# Train the Word2Vec Model
w2v_model = Word2Vec(tokenised_data, min_count = min_count, vector_size = vector_size, window = window, hs=hs,alpha=alpha,min_alpha=min_alpha,sample=sample,epochs=epochs)
print("Time taken to train word2vec model: " + str(time() - start_time))
w2v_model.save(word2vec_model_file) # can be used by classifier

Time taken to train word2vec model: 33.177834033966064


In [3]:
ref_pos = ['love', 'best', 'beautiful', 'great', 'cool', 'awesome', 'wonderful', 'brilliant', 'excellent',
                'fantastic']
ref_neg = ['bad', 'worst', 'stupid', 'disappointing', 'terrible', 'rubbish', 'boring', 'awful',
                'unwatchable', 'awkward']
neg_coefficient=0.5
pos_coefficient=0.5
def predict(tweet,neg_coefficient,pos_coefficient, model,true_ref_neg=ref_neg,true_ref_pos=ref_pos,count=0,confidence=0.5):
    sentence = np.zeros(model.vector_size)
    counter = 0
    cos_sim_bad, cos_sim_good = 0, 0
    for words in tweet:
        try:
            sentence += model.wv[words]  # np.array(list(model.wv[words]) + new_feature)
            counter += 1
        except:
            pass
    if counter != 0:
        sentence_vec = sentence / counter
    k_cur = min(len(true_ref_neg), len(true_ref_pos))
    for neg_word in true_ref_neg[:k_cur]:
        try:
            cos_sim_bad += np.dot(sentence_vec, model.wv[neg_word]) / (np.linalg.norm(sentence_vec) * np.linalg.norm(model.wv[neg_word]))
        except:     
            pass
    for pos_word in true_ref_pos[:k_cur]:
        try:
            cos_sim_good += np.dot(sentence_vec, model.wv[pos_word]) / (np.linalg.norm(sentence_vec) * np.linalg.norm(model.wv[pos_word]))
        except:
            pass
    if cos_sim_bad - cos_sim_good > confidence:
        # while count<10:
        #     print('in first if'+str(cos_sim_bad - cos_sim_good))
        return  0
    elif cos_sim_bad - cos_sim_good < -confidence:
        return  1
    else:
        if cos_sim_bad * neg_coefficient >= cos_sim_good * pos_coefficient:
            # if count<10:
            #     print('in sec if'+str([cos_sim_bad,cos_sim_good,pos_coefficient,neg_coefficient]))
            return  0
        else:
            return  1

In [59]:
def classify_result(tweets, neg_coefficient,pos_coefficient,w2v_mode,count=0):
    predictions=[]
    for t in range(len(tweets)):
        predict_result = predict(tweets[t], neg_coefficient,pos_coefficient,w2v_model,count=count)
        predictions.append(predict_result)
        neg_coefficient = predictions.count(0) / (predictions.count(1) + predictions.count(0))
        pos_coefficient = 1 - neg_coefficient
        count += 1
        
    ans = accuracy_score(true_label, predictions)
    # print(predictions.count(0))
    # print(predictions.count(1))
    # print(predictions[:10])
    count += 1
    return ans

In [63]:
start_time=time()

print(classify_result(tokenised_data, neg_coefficient,pos_coefficient,w2v_model))
print(time()-start_time)
print("vec size : "+str(vector_size),"epochs : "+str(epochs))

0.8180263157894737
32.79794430732727
vec size : 20 epochs : 30


0.8180263157894737
32.79794430732727
vec size : 20 epochs : 300.8180263157894737
32.79794430732727
vec size : 20 epochs : 30


0.823921052631579
29.009666919708252
vec size : 20 epochs : 60

epoch 60 dim 90</br>
0.8549736842105263</br>
34.335697412490845</br>

epoch 70 dim 90</br>
0.856078947368421</br>
30.640328407287598</br>

epoch 80 dim 90</br>
0.8548421052631578</br>
27.404643297195435</br>

epoch 90 dim 90</br>
0.8538157894736842</br>
31.939756870269775</br>

epoch 90 dim 100</br>
0.8587105263157895</br>
32.94704008102417</br>

epoch 100 dim 100</br>
0.8558684210526316</br>
29.760011911392212</br>

epoch 110 dim 200</br>
0.8533157894736842</br>
36.04504609107971</br>

epoch 100 dim 200 </br>
0.8519473684210527</br>
29.53914213180542</br>

In [166]:
w2v_model.vector_size

50

In [71]:
len(true_label)

38000

In [105]:
w2v_model.wv['awful']

array([ 0.0827959 , -0.4999122 ,  1.3272713 ,  0.6675552 ,  0.1212782 ,
       -0.52269405,  0.5355004 ,  0.18670893, -0.0178439 ,  0.80836487,
        0.09435534, -1.005927  , -0.34542146, -0.6518756 ,  0.3240503 ,
        0.5868083 ,  0.30133593, -0.613059  , -0.59222174, -0.67930746],
      dtype=float32)

### Test PLStream accuracy

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from time import time
from gensim.models import Word2Vec

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords

# f = pd.read_csv('./yelp_review_polarity_csv/test.csv', header=None)  # , encoding='ISO-8859-1'
f = pd.read_csv('train.csv', header=None)  # , encoding='ISO-8859-1'
f.columns = ["label", "review"]
f.loc[f['label'] == 1, 'label'] = 0
f.loc[f['label'] == 2, 'label'] = 1

true_label = f.label
yelp_review = f.review

start_time=time()
clean_data=[remove_stopwords(review)for review in yelp_review]#remove frequent words that does not carry sentiment
tokenised_data = [simple_preprocess(line, deacc=True) for line in clean_data] #remove punctuations and lowercase words also tokenise them
print('time taken: '+str(time()-start_time))
print(tokenised_data[:1])



time taken: 104.03294658660889
[['unfortunately', 'frustration', 'dr', 'goldberg', 'patient', 'repeat', 'experience', 've', 'doctors', 'nyc', 'good', 'doctor', 'terrible', 'staff', 'it', 'staff', 'simply', 'answers', 'phone', 'it', 'usually', 'takes', 'hours', 'repeated', 'calling', 'answer', 'who', 'time', 'wants', 'deal', 'it', 'run', 'problem', 'doctors', 'don', 'it', 'you', 'office', 'workers', 'patients', 'medical', 'needs', 'isn', 'answering', 'phone', 'it', 'work', 'aggravation', 'it', 'regret', 'feel', 'dr', 'goldberg', 'stars']]


In [4]:
m=Word2Vec([['good']*10,['bad']*10],vector_size=20,alpha=0.03,min_count=10,min_alpha=0.0007,window=4,hs=1,sample=0.00001,epochs=90)


vector_size=100
alpha=0.03
min_count=10
min_alpha=0.0007
window=4
hs=1
sample=0.00001
epochs=60

collector_size=2000
print(len(tokenised_data))
start=time()
for i in range(0,len(tokenised_data),collector_size):
    
    new_sentences=tokenised_data[i:collector_size+i]
    labels=true_label[i:collector_size+i]
    
    m.build_vocab(new_sentences, update=True)  # 1) update vocabulary
    m.train(new_sentences,  # 2) incremental training
                              total_examples=m.corpus_count,
                              epochs=m.epochs)
#     m = Word2Vec(tokenised_data, min_count = min_count, vector_size = vector_size, window = window, hs=hs,alpha=alpha,min_alpha=min_alpha,sample=sample,epochs=epochs)

    predictions=[]
    for sen in new_sentences:
        predict_result = predict(sen, neg_coefficient,pos_coefficient,m)
        predictions.append(predict_result)
        neg_coefficient = predictions.count(0) / (predictions.count(1) + predictions.count(0))
        pos_coefficient = 1 - neg_coefficient
        
    ans = accuracy_score(labels, predictions)
    # print(predictions.count(0))
    # print(predictions.count(1))
    # print(predictions[:10])
    print(ans)
print("time taken: "+str(time()-start))

560000
0.7495
0.7635
0.792
0.8
0.7895
0.7565
0.808
0.829
0.8315
0.823
0.7885
0.78
0.749
0.8185
0.7995
0.7515
0.806
0.7575
0.8145
0.8105
0.817
0.774
0.788
0.8325
0.8165
0.833
0.8475
0.8305
0.8445
0.819
0.8395
0.8085
0.7455
0.7085
0.7355
0.796
0.808
0.7565
0.8175
0.812
0.782
0.8165
0.818
0.835
0.787
0.826
0.7805
0.8385
0.787
0.792
0.8125
0.709
0.798
0.828
0.776
0.802
0.8335
0.7825
0.721
0.7975
0.7965
0.7805
0.79
0.788
0.795
0.687
0.7075
0.744
0.7445
0.835
0.765
0.805
0.799
0.84
0.743
0.7835
0.7095
0.712
0.782
0.77
0.796
0.786
0.79
0.771
0.773
0.772
0.821
0.8055
0.788
0.7745
0.7645
0.809
0.7895
0.802
0.7565
0.8305
0.841
0.8265
0.7765
0.793
0.8085
0.773
0.809
0.794
0.802
0.801
0.8125
0.8075
0.822
0.804
0.7855
0.8135
0.7645
0.7715
0.7735
0.758
0.761
0.792
0.7515
0.815
0.815
0.805
0.811
0.7835
0.792
0.795
0.8355
0.8315
0.8095
0.7895
0.801
0.7905
0.775
0.8025
0.805
0.8
0.826
0.7955
0.7725
0.788
0.794
0.795
0.791
0.741
0.7945
0.7755
0.8375
0.791
0.8125
0.7925
0.7325
0.797
0.82
0.814
0.7985
0.7

KeyboardInterrupt: 